In [21]:
import sys
import numpy as np
import pickle
import os
import glob
import networkx as nx
from tqdm import tqdm
import random

sys.path.append('../../')
from DataConstructor.notebooks.leafer import Leafer



seed = 42
random.seed(seed)
np.random.seed(seed)

In [22]:
G = nx.read_edgelist("../../TaxonomyEnrichment/data/noun/train.edgelist", delimiter="\t", create_using=nx.DiGraph)

In [23]:
wn = True

while True:
    try:
        cycle = nx.find_cycle(G)
        print(cycle)
        G.remove_edge(*cycle[0])
    except:
        break

if not wn:
    new_labels = {}
    for node in G.nodes():
        new_labels[node] = node + '.n.1'

    G_new = nx.relabel_nodes(G, new_labels)

    l = Leafer(G_new)
else:
    l = Leafer(G)

In [24]:
train, test = l.split_train_test(
    generation_depth=0,  # до какого уровня в топ. сортировке идти
    p=0.0,  # вероятность что подходящий случай уйдет в тест
    p_divide_leafs=0.5,
    # вероятность что листья поделим пополам трейн-тест
    # а не засунем целый случай в трейн или в тест
    min_to_test_rate=0.5,
    # минимальное количество доли вершин которых не было в
    # трейне чтобы поделить пополам на трейн-тест
    # то есть если 6\10 вершин были трейне то значит все 10 в трейн
    # если 5\10 были в трейне, то значит оставшиеся можем кинуть в тест
    weights=[0.00, 0.0, 0.0, 0.00, 0.00, 1.],
    # веса в соответствии
    # один ребенок, только листья, не только листья
    # триплеты с 2 родителями, триплеты такие что мидл нода имеет
    # 1 ребенка, предсказание родителя
    #p_parent=1
)

predict_hypernym 41363 41363


In [32]:
test_path = '../data/noun/test_nodes.pickle'
with open(test_path, 'rb') as f:
    test_nodes = pickle.load(f)

In [33]:
test_nodes[:5], train[:5]

([('unconventionality.n.02', ['unorthodoxy.n.03']),
  ('awnless_bromegrass.n.01', ['brome.n.01']),
  ('slenderness.n.01', ['insufficiency.n.03']),
  ('speedskater.n.01', ['ice-skater.n.01']),
  ('laminectomy.n.01', ['ablation.n.01'])],
 [{'children': 'dancing.n.01',
   'parents': 'diversion.n.01',
   'grandparents': None,
   'case': 'predict_hypernym'},
  {'children': 'bagel.n.01',
   'parents': 'bun.n.01',
   'grandparents': None,
   'case': 'predict_hypernym'},
  {'children': 'shopping_bag.n.01',
   'parents': 'bag.n.01',
   'grandparents': None,
   'case': 'predict_hypernym'},
  {'children': 'pteridology.n.01',
   'parents': 'botany.n.02',
   'grandparents': None,
   'case': 'predict_hypernym'},
  {'children': 'house_wren.n.01',
   'parents': 'wren.n.02',
   'grandparents': None,
   'case': 'predict_hypernym'}])

In [28]:
test = []

for child, parents in test_nodes:
    elem = {}
    elem["children"] = child
    elem["grandparents"] = None
    if len(parents) == 1:
        elem["parents"] = parents[0]
        elem["case"] = "predict_hypernym"
    else:
        elem["parents"] = parents
        elem["case"] = "predict_multiple_hypernyms"

    test.append(elem)

In [29]:
test[:5]

[{'children': 'unconventionality.n.02',
  'grandparents': None,
  'parents': 'unorthodoxy.n.03',
  'case': 'predict_hypernym'},
 {'children': 'awnless_bromegrass.n.01',
  'grandparents': None,
  'parents': 'brome.n.01',
  'case': 'predict_hypernym'},
 {'children': 'slenderness.n.01',
  'grandparents': None,
  'parents': 'insufficiency.n.03',
  'case': 'predict_hypernym'},
 {'children': 'speedskater.n.01',
  'grandparents': None,
  'parents': 'ice-skater.n.01',
  'case': 'predict_hypernym'},
 {'children': 'laminectomy.n.01',
  'grandparents': None,
  'parents': 'ablation.n.01',
  'case': 'predict_hypernym'}]

In [30]:
train[:5]

[{'children': 'dancing.n.01',
  'parents': 'diversion.n.01',
  'grandparents': None,
  'case': 'predict_hypernym'},
 {'children': 'bagel.n.01',
  'parents': 'bun.n.01',
  'grandparents': None,
  'case': 'predict_hypernym'},
 {'children': 'shopping_bag.n.01',
  'parents': 'bag.n.01',
  'grandparents': None,
  'case': 'predict_hypernym'},
 {'children': 'pteridology.n.01',
  'parents': 'botany.n.02',
  'grandparents': None,
  'case': 'predict_hypernym'},
 {'children': 'house_wren.n.01',
  'parents': 'wren.n.02',
  'grandparents': None,
  'case': 'predict_hypernym'}]

In [31]:
with open('../data/noun/test_hypernyms.pickle', 'wb') as f:
    pickle.dump(test, f)

with open('../data/noun/train_hypernyms.pickle', 'wb') as f:
    pickle.dump(train, f)